# Ejercicio

- Vamos a hacer un algoritmo que cada 200 días rebalancee todos los activos para tener el mismo porcertaje.
- Estamos en modo backtesting: calculamos primero todas las allocations, las insertamos y ejecutamos el backtesting.

1. Descarga el maestro de valores.
2. Descarga todos los datos para cada ticker del maestro. Baja solo el close.
3. Con las series close, crea un datafame donde tengas como columnas los tickers y filas las fechas.
4. Recorre este dataframe cada 200 filas y crea una lista de allocations con valor 1/n_activos. 
5. Envía el post de estos allocations.
6. Usa la API para obtener todas las allocations introducidas.
7. Usa la API para ejecutar el backtesting.
8. Elimina todas las allocations.
9. Refactoriza el código en una clase para el algoritmo y otra para un handler del API.
10. Ejecutalo desde la terminal.

In [1]:
import pandas as pd
import requests, json

In [7]:
url_base = 'https://miax-gateway-jog4ew3z3q-ew.a.run.app'

competi = 'mia_10'
user_key = ''


def get_ticker_master():
    url = f'{url_base}/data/ticker_master'
    params = {
        'competi': competi,
        'market': 'IBEX',
        'key': user_key
    }
    response = requests.get(url, params)
    tk_master = response.json()
    maestro_df = pd.DataFrame(tk_master['master'])
    return maestro_df


def get_close_data(tck):
    url2 = f'{url_base}/data/time_series'
    params = {
        'market': 'IBEX',
        'key': user_key,
        'ticker': tck,
        'close': True
    }
    response = requests.get(url2, params)
    tk_data = response.json()
    series_data = pd.read_json(tk_data, typ='series')
    return series_data


In [21]:
get_ticker_master()

,ticker,start_date,end_date,n_days
0,ABE,2010-01-04T00:00:00,2018-05-09T00:00:00,2136
1,ABG,2010-01-04T00:00:00,2012-10-26T00:00:00,725
2,ABG.P_0,2012-10-26T00:00:00,2013-07-01T00:00:00,171
3,ABG.P_1,2014-06-23T00:00:00,2015-11-27T00:00:00,369
4,ACS,2010-01-04T00:00:00,,
...,...,...,...,...
62,TEF,2010-01-04T00:00:00,,
63,TL5,2010-01-04T00:00:00,2020-06-22T00:00:00,2677
64,TRE,2010-01-04T00:00:00,2019-06-24T00:00:00,2423
65,VIS,2016-06-21T00:00:00,2021-12-17T00:00:00,1408


In [11]:
get_close_data('SAN')

2010-01-04    4.400382
2010-01-05    4.449025
2010-01-06    4.480831
2010-01-07    4.460251
2010-01-08    4.475218
                ...   
2023-01-20    3.097000
2023-01-23    3.095500
2023-01-24    3.104000
2023-01-25    3.093500
2023-01-26    3.194000
Length: 3347, dtype: float64

In [12]:
maestro = get_ticker_master()

In [16]:
data_close_all = {}
for _, row in maestro.iterrows():
    tck = row.ticker
    print(f"Download: {tck}", end=' ')
    close_data = get_close_data(tck)
    data_close_all[tck] = close_data

Dowload: ABE Dowload: ABG Dowload: ABG.P_0 Dowload: ABG.P_1 Dowload: ACS Dowload: ACX Dowload: ACX_0 Dowload: AENA Dowload: ALM Dowload: AMS Dowload: ANA Dowload: ANA_0 Dowload: ANE Dowload: BBVA Dowload: BKIA Dowload: BKIA_0 Dowload: BKT Dowload: BME Dowload: BTO_1 Dowload: CABK Dowload: CIE Dowload: CLNX Dowload: COL Dowload: CRI Dowload: DIA Dowload: EBRO_0 Dowload: EBRO_1 Dowload: ELE Dowload: ELE_0 Dowload: ENC Dowload: ENG Dowload: EVA Dowload: FCC Dowload: FDR Dowload: FER Dowload: GRF Dowload: IAG Dowload: IBE Dowload: IBLA Dowload: IBR Dowload: IDR Dowload: ITX Dowload: JAZ Dowload: MAP Dowload: MAS Dowload: MEL Dowload: MRL Dowload: MTS Dowload: NTGY Dowload: OHL Dowload: PHM Dowload: POP Dowload: REE Dowload: REP Dowload: ROVI Dowload: SAB Dowload: SAN Dowload: SCYR Dowload: SCYR_1 Dowload: SGRE Dowload: SGRE_0 Dowload: SLR Dowload: TEF Dowload: TL5 Dowload: TRE Dowload: VIS Dowload: VIS_0 

In [19]:
df_close = pd.DataFrame(data_close_all)

In [23]:
df_close

,ABE,ABG,ABG.P_0,ABG.P_1,ACS,ACX,ACX_0,AENA,ALM,AMS,...,SCYR,SCYR_1,SGRE,SGRE_0,SLR,TEF,TL5,TRE,VIS,VIS_0
2010-01-04,7.079902,3.9889,NaN,NaN,15.285421,NaN,9.722429,NaN,NaN,NaN,...,NaN,5.381871,NaN,9.364632,NaN,8.067411,6.904748,29.693987,NaN,NaN
2010-01-05,7.110934,3.9813,NaN,NaN,15.553961,NaN,9.797462,NaN,NaN,NaN,...,NaN,5.496363,NaN,9.544552,NaN,8.055200,6.938567,30.546259,NaN,NaN
2010-01-06,7.137689,4.0098,NaN,NaN,15.585929,NaN,9.755071,NaN,NaN,NaN,...,NaN,5.552471,NaN,9.624838,NaN,7.996180,6.904748,29.882358,NaN,NaN
2010-01-07,7.077627,4.0368,NaN,NaN,15.430347,NaN,9.807211,NaN,NaN,NaN,...,NaN,5.629802,NaN,9.666842,NaN,7.916809,6.993000,30.283471,NaN,NaN
2010-01-08,7.008919,3.9922,NaN,NaN,15.767087,NaN,9.813826,NaN,NaN,NaN,...,NaN,6.026540,NaN,9.708928,NaN,7.778417,7.064299,30.187564,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-20,NaN,NaN,NaN,NaN,27.580000,9.8522,NaN,137.50,NaN,56.46,...,2.648,NaN,18.05,NaN,19.365,3.571000,NaN,NaN,NaN,NaN
2023-01-23,NaN,NaN,NaN,NaN,26.960000,9.9250,NaN,136.75,NaN,56.64,...,2.688,NaN,18.05,NaN,19.395,3.586000,NaN,NaN,NaN,NaN
2023-01-24,NaN,NaN,NaN,NaN,26.980000,9.9250,NaN,137.40,NaN,57.14,...,2.732,NaN,18.05,NaN,19.390,3.542000,NaN,NaN,NaN,NaN
2023-01-25,NaN,NaN,NaN,NaN,26.960000,9.9000,NaN,141.30,NaN,58.52,...,2.780,NaN,18.05,NaN,19.410,3.482000,NaN,NaN,NaN,NaN


In [25]:
df_close.iloc[::200]

,ABE,ABG,ABG.P_0,ABG.P_1,ACS,ACX,ACX_0,AENA,ALM,AMS,...,SCYR,SCYR_1,SGRE,SGRE_0,SLR,TEF,TL5,TRE,VIS,VIS_0
2010-01-04,7.079902,3.9889,NaN,NaN,15.285421,NaN,9.722429,NaN,NaN,NaN,...,NaN,5.381871,NaN,9.364632,NaN,8.067411,6.904748,29.693987,NaN,NaN
2010-10-13,6.765575,3.2939,NaN,NaN,16.977973,NaN,8.717492,NaN,NaN,NaN,...,NaN,3.429484,NaN,3.634791,NaN,8.181211,5.930403,32.369035,NaN,NaN
2011-07-26,7.239431,3.4268,NaN,NaN,14.405474,NaN,8.140643,NaN,NaN,11.557136,...,NaN,4.147988,NaN,4.201756,NaN,7.314817,4.567019,23.496085,NaN,NaN
2012-05-07,7.015835,2.0069,NaN,NaN,7.135125,NaN,6.156012,NaN,NaN,13.114002,...,NaN,0.904229,NaN,1.585632,NaN,5.369255,2.788017,22.870974,NaN,NaN
2013-02-14,8.297804,NaN,1.6716,NaN,10.178788,NaN,6.008558,NaN,NaN,17.346107,...,NaN,1.264132,NaN,NaN,NaN,5.185869,4.309079,31.438810,NaN,32.019973
2013-11-26,10.886403,NaN,NaN,NaN,14.009506,NaN,7.470395,NaN,NaN,24.415016,...,NaN,2.985257,NaN,NaN,NaN,6.499800,6.828593,32.131083,NaN,32.746258
2014-09-10,12.111039,NaN,NaN,4.2003,20.009850,NaN,NaN,NaN,NaN,25.486592,...,NaN,3.419970,7.618121,NaN,NaN,6.735082,7.523191,36.585651,NaN,36.366076
2015-06-25,12.674988,NaN,NaN,3.0940,19.119256,9.192037,NaN,81.892023,NaN,33.494629,...,NaN,2.802948,11.924192,NaN,NaN,7.959738,9.930431,40.962452,NaN,NaN
2016-04-06,12.277849,NaN,NaN,NaN,17.193370,6.832424,NaN,102.782845,NaN,35.219335,...,NaN,1.272589,13.756527,NaN,NaN,5.566788,8.183288,22.299165,NaN,NaN
2017-01-12,12.619489,NaN,NaN,NaN,20.643214,8.936819,NaN,121.518701,NaN,41.425651,...,NaN,NaN,16.181267,NaN,NaN,6.072642,9.280896,35.785751,42.059675,NaN


In [45]:
for date, data in df_close.iloc[::200].iterrows():
    print(date)
    tcks_activos = data.dropna().index
    alloc = 1/tcks_activos.shape[0]
    allocations_to_sent = [
        {'ticker': tck, 'alloc': alloc}
        for tck in tcks_activos
    ]  

2010-01-04 00:00:00
2010-10-13 00:00:00
2011-07-26 00:00:00
2012-05-07 00:00:00
2013-02-14 00:00:00
2013-11-26 00:00:00
2014-09-10 00:00:00
2015-06-25 00:00:00
2016-04-06 00:00:00
2017-01-12 00:00:00
2017-10-24 00:00:00
2018-08-08 00:00:00
2019-05-23 00:00:00
2020-03-03 00:00:00
2020-12-11 00:00:00
2021-09-23 00:00:00
2022-07-05 00:00:00
